## Web scraper to take data from CDIP at specified year/month:

In [ ]:
# Loosely based on tutorial at: https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059

In [5]:
#Import files
import requests
import lxml.html as lh
import pandas as pd
print("Done.")

Done.


In [11]:
# Format should be appending year, month
# Example: Sept. 2019 -> 201909
def create_url_string(year, month):
    url='https://cdip.ucsd.edu/themes/cdip?tz=UTC&numcolorbands=10&palette=cdip_classic&zoom=auto&ll_fmt=dm&high=6.096&r=999&un=1&pb=1&d2=p70&u2=s:201:st:1:v:parameter:dt:'
    if 2014 <= int(year) <= 2019:
        year = year
    if 1 <= int(month) <= 12 and len(month) == 2:
        month = month
    url += year
    url += month
    return url 

## Choose the year and month that you want here: 

In [12]:
url = create_url_string('2019', '09')
print(url)

https://cdip.ucsd.edu/themes/cdip?tz=UTC&numcolorbands=10&palette=cdip_classic&zoom=auto&ll_fmt=dm&high=6.096&r=999&un=1&pb=1&d2=p70&u2=s:201:st:1:v:parameter:dt:201909


In [20]:
# Create a handle, page, to handle the contents of the website
page = requests.get(url)

# Store the contents of the website under doc
doc = lh.fromstring(page.content)

# Parse data stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

# Check the length of the first 12 rows (should all be same)
print([len(T) for T in tr_elements[:14]])

# Need to drop the first 3 rows since they aren't in the table
tr_elements = tr_elements[3:]
[len(T) for T in tr_elements[:14]]

[2, 2, 1, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]


[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]

In [95]:
# Parse the first row as the header
tr_elements = doc.xpath('//tr')

# Create empty list
headers = []
i = 0

# For each row, store each first element (header) and an empty list
for t in tr_elements[3]:
    i+=1
    name=t.text_content()
    print(name)
    headers.append(name)

print(headers)

Date(UTC)
Hs(ft)
Tp(s)
Dp(deg)
Ta(s)
SST(F)
Air temp(F)
['Date(UTC)', 'Hs(ft)', 'Tp(s)', 'Dp(deg)', 'Ta(s)', 'SST(F)', 'Air temp(F)']


In [113]:
def parse_rows(row_number):
    name_string = ""
    name_list = []
    for t in tr_elements[row_number]:
        name = t.text_content()
        for i in name: 
            name_string += i
    #print(name_string)

    # Create a string from the values
    name_string = name_string.split(" ")
    
    # Remove all spaces from the list
    for i in name_string: 
        if len(i) > 0:
            name_list.append(i)
      
    # Ensure that time stays with 'Date (UTC)'' header
    name_list[0] = name_list[0] + " " +  name_list[1]
    name_list.pop(1)
    
    return name_list

In [114]:
# The first row that cna be parsed starts at 4
name_list = parse_rows(4)
print(name_list)

['2019-09-30 23:32', '3.54', '7.69', '285', '6.01', '71.2', '68.4']


In [125]:
# Create a Pandas dataframe: 
data_list = []

#Since out first row is the header, data is stored on the second row onwards
for j in range(4, len(tr_elements)):
    data = parse_rows(j)
    data_list.append(data)

df_data = pd.DataFrame(data_list, columns=headers)
print(df_data[:10])

          Date(UTC) Hs(ft) Tp(s) Dp(deg) Ta(s) SST(F) Air temp(F)
0  2019-09-30 23:32   3.54  7.69     285  6.01   71.2        68.4
1  2019-09-30 23:02   3.41  7.69     282  6.16   71.2        68.2
2  2019-09-30 22:32   3.48  7.69     282  6.11   71.2        67.6
3  2019-09-30 22:02   3.41  7.14     281  6.14   71.4        67.1
4  2019-09-30 21:32   3.41  8.33     285  6.13   71.2        66.9
5  2019-09-30 21:02   3.28  8.33     284  6.15   70.9        66.7
6  2019-09-30 20:32   3.28  7.69     284  6.04   70.9        66.4
7  2019-09-30 20:02   3.38  7.14     284  5.82   70.9        65.8
8  2019-09-30 19:32   3.15  7.69     284  5.85   70.7        65.3
9  2019-09-30 19:02   3.18  5.88     285  5.60   70.7        65.1


## How to use this data: 

### 1. If you want to find the data corresponding to the closest time as the surf session that you're interested in: 

In [196]:
def find_closest_time(date, time_utc):
    time_list = []
    
    # Iterate over each date
    for x in df_data['Date(UTC)']:
        # Choose all timestamps within the same hour
        a = x.split(" ")
        if date == a[0]:
            a = str(a[1]).split(':')
            y = str(time_utc).split(":")
            if a[0] == y[0]:
                time_string = str(a[0]) + ":" + str(a[1])
                time_list.append(time_string)
    return time_list

In [197]:
print(df_data['Date(UTC)'][0])
x = df_data['Date(UTC)'][0].split(" ")
print(x)
x = str(x).split(':')
print(x)

2019-09-30 23:32
['2019-09-30', '23:32']
["['2019-09-30', '23", "32']"]


In [200]:
date = '2019-09-30'
time_list = find_closest_time(date, '19:05')
print(time_list)

['19:32', '19:02']


### Now you can do something like compute the average wave height for the one hour time interval:

In [216]:
print(df_data['Date(UTC)'].size)
length = df_data['Date(UTC)'].size

1440


In [223]:
wave_height_list = []
for time in time_list: 
    date_data = date + " " + time
    for i in range(0, length):
        if df_data['Date(UTC)'][i] == date_data:
            #print(df_data['Hs(ft)'][i])
            wave_height_list.append(int(df_data['Hs(ft)'][i])
print(wave_height_list)

ValueError: invalid literal for int() with base 10: '3.15'